In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
#ds.close()
ds=xr.open_dataset(dir_data+'ar_vars_geomeans_era5.nc')
ivt_concat_era5_variable=ds['ivt_concat_era5_variable']
iwv_concat_era5_variable=ds['iwv_concat_era5_variable']
s_concat_era5_variable=ds['s_concat_era5_variable']
mfc_concat_era5_variable=ds['mfc_concat_era5_variable']

ivt_concat_era5_fixed=ds['ivt_concat_era5_fixed']
iwv_concat_era5_fixed=ds['iwv_concat_era5_fixed']
s_concat_era5_fixed=ds['s_concat_era5_fixed']
mfc_concat_era5_fixed=ds['mfc_concat_era5_fixed']

ivt_concat_era5_fixed_poleward=ds['ivt_concat_era5_fixed_poleward']
iwv_concat_era5_fixed_poleward=ds['iwv_concat_era5_fixed_poleward']
s_concat_era5_fixed_poleward=ds['s_concat_era5_fixed_poleward']
mfc_concat_era5_fixed_poleward=ds['mfc_concat_era5_fixed_poleward']


ds=xr.open_dataset(dir_data+'ar_vars_geomeans_merra2.nc')
ivt_concat_merra2_variable=ds['ivt_concat_merra2_variable']
iwv_concat_merra2_variable=ds['iwv_concat_merra2_variable']
s_concat_merra2_variable=ds['s_concat_merra2_variable']
mfc_concat_merra2_variable=ds['mfc_concat_merra2_variable']

ivt_concat_merra2_fixed=ds['ivt_concat_merra2_fixed']
iwv_concat_merra2_fixed=ds['iwv_concat_merra2_fixed']
s_concat_merra2_fixed=ds['s_concat_merra2_fixed']
mfc_concat_merra2_fixed=ds['mfc_concat_merra2_fixed']

ivt_concat_merra2_fixed_poleward=ds['ivt_concat_merra2_fixed_poleward']
iwv_concat_merra2_fixed_poleward=ds['iwv_concat_merra2_fixed_poleward']
s_concat_merra2_fixed_poleward=ds['s_concat_merra2_fixed_poleward']
mfc_concat_merra2_fixed_poleward=ds['mfc_concat_merra2_fixed_poleward']


ds=xr.open_dataset(dir_data+'ar_vars_geomeans_jra55.nc')
ivt_concat_jra55_variable=ds['ivt_concat_jra55_variable']
iwv_concat_jra55_variable=ds['iwv_concat_jra55_variable']
s_concat_jra55_variable=ds['s_concat_jra55_variable']

ivt_concat_jra55_fixed=ds['ivt_concat_jra55_fixed']
iwv_concat_jra55_fixed=ds['iwv_concat_jra55_fixed']
s_concat_jra55_fixed=ds['s_concat_jra55_fixed']

ivt_concat_jra55_fixed_poleward=ds['ivt_concat_jra55_fixed_poleward']
iwv_concat_jra55_fixed_poleward=ds['iwv_concat_jra55_fixed_poleward']
s_concat_jra55_fixed_poleward=ds['s_concat_jra55_fixed_poleward']

In [ ]:
yrs=np.arange(1980,2024)

lons_west=[x for x in ivt_concat_jra55_fixed.lon.values if x<-90]
lons_east=[x for x in ivt_concat_jra55_fixed.lon.values if x>=90]
sel_west=ivt_concat_jra55_fixed.sel(lon=lons_west)
sel_east=ivt_concat_jra55_fixed.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,ivt_concat_jra55_fixed,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_ivt_fixed=interp.sel(lon=lons_center)


sel_west=ivt_concat_jra55_variable.sel(lon=lons_west)
sel_east=ivt_concat_jra55_variable.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,ivt_concat_jra55_variable,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_ivt_variable=interp.sel(lon=lons_center)


sel_west=ivt_concat_jra55_fixed_poleward.sel(lon=lons_west)
sel_east=ivt_concat_jra55_fixed_poleward.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,ivt_concat_jra55_fixed_poleward,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_ivt_fixed_poleward=interp.sel(lon=lons_center)

####

sel_west=iwv_concat_jra55_fixed.sel(lon=lons_west)
sel_east=iwv_concat_jra55_fixed.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,iwv_concat_jra55_fixed,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_iwv_fixed=interp.sel(lon=lons_center)


sel_west=iwv_concat_jra55_variable.sel(lon=lons_west)
sel_east=iwv_concat_jra55_variable.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,iwv_concat_jra55_variable,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_iwv_variable=interp.sel(lon=lons_center)


sel_west=iwv_concat_jra55_fixed_poleward.sel(lon=lons_west)
sel_east=iwv_concat_jra55_fixed_poleward.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,iwv_concat_jra55_fixed_poleward,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_iwv_fixed_poleward=interp.sel(lon=lons_center)

#####

lons_west=[x for x in s_concat_jra55_fixed.lon1.values if x<-90]
lons_east=[x for x in s_concat_jra55_fixed.lon1.values if x>=90]
sel_west=s_concat_jra55_fixed.sel(lon1=lons_west)
sel_east=s_concat_jra55_fixed.sel(lon1=lons_east)
sel_west['lon1']=[x+360. for x in lons_west]
sel_east['lon1']=[x-360. for x in lons_east]

final=xr.concat([sel_east,s_concat_jra55_fixed,sel_west],dim='lon1')
dk=xr.Dataset()
dk['var']=(('time','lat','lon'),final.values)
dk.coords['time']=yrs
dk.coords['lat']=final.lat1.values
dk.coords['lon']=final.lon1.values
final1=dk['var']

interp=final1.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_s_fixed=interp.sel(lon=lons_center)


sel_west=s_concat_jra55_variable.sel(lon1=lons_west)
sel_east=s_concat_jra55_variable.sel(lon1=lons_east)
sel_west['lon1']=[x+360. for x in lons_west]
sel_east['lon1']=[x-360. for x in lons_east]
final=xr.concat([sel_east,s_concat_jra55_variable,sel_west],dim='lon1')

dk=xr.Dataset()
dk['var']=(('time','lat','lon'),final.values)
dk.coords['time']=yrs
dk.coords['lat']=final.lat1.values
dk.coords['lon']=final.lon1.values
final1=dk['var']

interp=final1.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_s_variable=interp.sel(lon=lons_center)


sel_west=s_concat_jra55_fixed_poleward.sel(lon1=lons_west)
sel_east=s_concat_jra55_fixed_poleward.sel(lon1=lons_east)
sel_west['lon1']=[x+360. for x in lons_west]
sel_east['lon1']=[x-360. for x in lons_east]
final=xr.concat([sel_east,s_concat_jra55_fixed_poleward,sel_west],dim='lon1')

dk=xr.Dataset()
dk['var']=(('time','lat','lon'),final.values)
dk.coords['time']=yrs
dk.coords['lat']=final.lat1.values
dk.coords['lon']=final.lon1.values
final1=dk['var']

interp=final1.interp(lat=ivt_concat_era5_fixed.lat.values,lon=ivt_concat_era5_fixed.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_s_fixed_poleward=interp.sel(lon=lons_center)

In [ ]:
yrs=np.arange(1980,2024,1)
slopes_array_ivt_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_ivt_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

slopes_array_iwv_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_iwv_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

slopes_array_s_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_jra55_variable=xr.zeros_like(s_concat_era5_variable[0,:,:])
slopes_array_s_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_jra55_fixed=xr.zeros_like(s_concat_era5_variable[0,:,:])
slopes_array_s_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_s_jra55_fixed_poleward=xr.zeros_like(s_concat_era5_variable[0,:,:])

slopes_array_mfc_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#slopes_array_mfc_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#slopes_array_mfc_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
slopes_array_mfc_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#slopes_array_mfc_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])



sigs_array_ivt_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_ivt_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

sigs_array_iwv_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_iwv_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

sigs_array_s_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_jra55_variable=xr.zeros_like(s_concat_era5_variable[0,:,:])
sigs_array_s_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_jra55_fixed=xr.zeros_like(s_concat_era5_variable[0,:,:])
sigs_array_s_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_s_jra55_fixed_poleward=xr.zeros_like(s_concat_era5_variable[0,:,:])

sigs_array_mfc_era5_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_merra2_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#sigs_array_mfc_jra55_variable=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_era5_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_merra2_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#sigs_array_mfc_jra55_fixed=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_era5_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
sigs_array_mfc_merra2_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])
#sigs_array_mfc_jra55_fixed_poleward=xr.zeros_like(ivt_concat_era5_variable[0,:,:])

size=20
for j in range(slopes_array_ivt_era5_variable.lat.size):
    print(j)
    for k in range(slopes_array_ivt_era5_variable.lon.size):
        time_series=ivt_concat_era5_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5_variable[j,k]=r[0]
            sigs_array_ivt_era5_variable[j,k]=r[3]
        else:
            slopes_array_ivt_era5_variable[j,k]=np.nan
            sigs_array_ivt_era5_variable[j,k]=np.nan
        
        time_series=iwv_concat_era5_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5_variable[j,k]=r[0]
            sigs_array_iwv_era5_variable[j,k]=r[3]
        else:
            slopes_array_iwv_era5_variable[j,k]=np.nan
            sigs_array_iwv_era5_variable[j,k]=np.nan
        
        time_series=s_concat_era5_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5_variable[j,k]=r[0]
            sigs_array_s_era5_variable[j,k]=r[3]
        else:
            slopes_array_s_era5_variable[j,k]=np.nan
            sigs_array_s_era5_variable[j,k]=np.nan

        time_series=mfc_concat_era5_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5_variable[j,k]=r[0]
            sigs_array_mfc_era5_variable[j,k]=r[3]
        else:
            slopes_array_mfc_era5_variable[j,k]=np.nan
            sigs_array_mfc_era5_variable[j,k]=np.nan
        
        time_series=ivt_concat_era5_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5_fixed[j,k]=r[0]
            sigs_array_ivt_era5_fixed[j,k]=r[3]
        else:
            slopes_array_ivt_era5_fixed[j,k]=np.nan
            sigs_array_ivt_era5_fixed[j,k]=np.nan
        
        time_series=iwv_concat_era5_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5_fixed[j,k]=r[0]
            sigs_array_iwv_era5_fixed[j,k]=r[3]
        else:
            slopes_array_iwv_era5_fixed[j,k]=np.nan
            sigs_array_iwv_era5_fixed[j,k]=np.nan
        
        time_series=s_concat_era5_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5_fixed[j,k]=r[0]
            sigs_array_s_era5_fixed[j,k]=r[3]
        else:
            slopes_array_s_era5_fixed[j,k]=np.nan
            sigs_array_s_era5_fixed[j,k]=np.nan
        
        time_series=mfc_concat_era5_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5_fixed[j,k]=r[0]
            sigs_array_mfc_era5_fixed[j,k]=r[3]
        else:
            slopes_array_mfc_era5_fixed[j,k]=np.nan
            sigs_array_mfc_era5_fixed[j,k]=np.nan
        
        time_series=ivt_concat_era5_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5_fixed_poleward[j,k]=r[0]
            sigs_array_ivt_era5_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_ivt_era5_fixed_poleward[j,k]=np.nan
            sigs_array_ivt_era5_fixed_poleward[j,k]=np.nan
        
        time_series=iwv_concat_era5_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5_fixed_poleward[j,k]=r[0]
            sigs_array_iwv_era5_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_iwv_era5_fixed_poleward[j,k]=np.nan
            sigs_array_iwv_era5_fixed_poleward[j,k]=np.nan

        time_series=s_concat_era5_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5_fixed_poleward[j,k]=r[0]
            sigs_array_s_era5_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_s_era5_fixed_poleward[j,k]=np.nan
            sigs_array_s_era5_fixed_poleward[j,k]=np.nan
        
        time_series=mfc_concat_era5_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5_fixed_poleward[j,k]=r[0]
            sigs_array_mfc_era5_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_mfc_era5_fixed_poleward[j,k]=np.nan
            sigs_array_mfc_era5_fixed_poleward[j,k]=np.nan
        
        
        #############
        
        time_series=ivt_concat_merra2_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_merra2_variable[j,k]=r[0]
            sigs_array_ivt_merra2_variable[j,k]=r[3]
        else:
            slopes_array_ivt_merra2_variable[j,k]=np.nan
            sigs_array_ivt_merra2_variable[j,k]=np.nan
        
        time_series=iwv_concat_merra2_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_merra2_variable[j,k]=r[0]
            sigs_array_iwv_merra2_variable[j,k]=r[3]
        else:
            slopes_array_iwv_merra2_variable[j,k]=np.nan
            sigs_array_iwv_merra2_variable[j,k]=np.nan
        
        time_series=s_concat_merra2_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_merra2_variable[j,k]=r[0]
            sigs_array_s_merra2_variable[j,k]=r[3]
        else:
            slopes_array_s_merra2_variable[j,k]=np.nan
            sigs_array_s_merra2_variable[j,k]=np.nan
        
        time_series=mfc_concat_merra2_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_merra2_variable[j,k]=r[0]
            sigs_array_mfc_merra2_variable[j,k]=r[3]
        else:
            slopes_array_mfc_merra2_variable[j,k]=np.nan
            sigs_array_mfc_merra2_variable[j,k]=np.nan
        
        time_series=ivt_concat_merra2_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_merra2_fixed[j,k]=r[0]
            sigs_array_ivt_merra2_fixed[j,k]=r[3]
        else:
            slopes_array_ivt_merra2_fixed[j,k]=np.nan
            sigs_array_ivt_merra2_fixed[j,k]=np.nan
        
        time_series=iwv_concat_merra2_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_merra2_fixed[j,k]=r[0]
            sigs_array_iwv_merra2_fixed[j,k]=r[3]
        else:
            slopes_array_iwv_merra2_fixed[j,k]=np.nan
            sigs_array_iwv_merra2_fixed[j,k]=np.nan
        
        time_series=s_concat_merra2_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_merra2_fixed[j,k]=r[0]
            sigs_array_s_merra2_fixed[j,k]=r[3]
        else:
            slopes_array_s_merra2_fixed[j,k]=np.nan
            sigs_array_s_merra2_fixed[j,k]=np.nan
        
        time_series=mfc_concat_merra2_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_merra2_fixed[j,k]=r[0]
            sigs_array_mfc_merra2_fixed[j,k]=r[3]
        else:
            slopes_array_mfc_merra2_fixed[j,k]=np.nan
            sigs_array_mfc_merra2_fixed[j,k]=np.nan
        
        time_series=ivt_concat_merra2_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_merra2_fixed_poleward[j,k]=r[0]
            sigs_array_ivt_merra2_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_ivt_merra2_fixed_poleward[j,k]=np.nan
            sigs_array_ivt_merra2_fixed_poleward[j,k]=np.nan
        
        time_series=iwv_concat_merra2_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_merra2_fixed_poleward[j,k]=r[0]
            sigs_array_iwv_merra2_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_iwv_merra2_fixed_poleward[j,k]=np.nan
            sigs_array_iwv_merra2_fixed_poleward[j,k]=np.nan
        
        time_series=s_concat_merra2_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_merra2_fixed_poleward[j,k]=r[0]
            sigs_array_s_merra2_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_s_merra2_fixed_poleward[j,k]=np.nan
            sigs_array_s_merra2_fixed_poleward[j,k]=np.nan
            
        time_series=mfc_concat_merra2_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_merra2_fixed_poleward[j,k]=r[0]
            sigs_array_mfc_merra2_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_mfc_merra2_fixed_poleward[j,k]=np.nan
            sigs_array_mfc_merra2_fixed_poleward[j,k]=np.nan
        
        
        #############
        
        time_series=fullres_ivt_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_jra55_variable[j,k]=r[0]
            sigs_array_ivt_jra55_variable[j,k]=r[3]
        else:
            slopes_array_ivt_jra55_variable[j,k]=np.nan
            sigs_array_ivt_jra55_variable[j,k]=np.nan
        
        time_series=fullres_iwv_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_jra55_variable[j,k]=r[0]
            sigs_array_iwv_jra55_variable[j,k]=r[3]
        else:
            slopes_array_iwv_jra55_variable[j,k]=np.nan
            sigs_array_iwv_jra55_variable[j,k]=np.nan
        
        time_series=fullres_ivt_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_jra55_fixed[j,k]=r[0]
            sigs_array_ivt_jra55_fixed[j,k]=r[3]
        else:
            slopes_array_ivt_jra55_fixed[j,k]=np.nan
            sigs_array_ivt_jra55_fixed[j,k]=np.nan
        
        time_series=fullres_iwv_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_jra55_fixed[j,k]=r[0]
            sigs_array_iwv_jra55_fixed[j,k]=r[3]
        else:
            slopes_array_iwv_jra55_fixed[j,k]=np.nan
            sigs_array_iwv_jra55_fixed[j,k]=np.nan
            
        time_series=fullres_ivt_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_jra55_fixed_poleward[j,k]=r[0]
            sigs_array_ivt_jra55_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_ivt_jra55_fixed_poleward[j,k]=np.nan
            sigs_array_ivt_jra55_fixed_poleward[j,k]=np.nan
        
        time_series=fullres_iwv_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_jra55_fixed_poleward[j,k]=r[0]
            sigs_array_iwv_jra55_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_iwv_jra55_fixed_poleward[j,k]=np.nan
            sigs_array_iwv_jra55_fixed_poleward[j,k]=np.nan
        
        time_series=fullres_s_variable[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_jra55_variable[j,k]=r[0]
            sigs_array_s_jra55_variable[j,k]=r[3]
        else:
            slopes_array_s_jra55_variable[j,k]=np.nan
            sigs_array_s_jra55_variable[j,k]=np.nan
        
        time_series=fullres_s_fixed[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_jra55_fixed[j,k]=r[0]
            sigs_array_s_jra55_fixed[j,k]=r[3]
        else:
            slopes_array_s_jra55_fixed[j,k]=np.nan
            sigs_array_s_jra55_fixed[j,k]=np.nan
        
        time_series=fullres_s_fixed_poleward[:,j,k].values.tolist()
        if len([x for x in time_series if x>-1])>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_jra55_fixed_poleward[j,k]=r[0]
            sigs_array_s_jra55_fixed_poleward[j,k]=r[3]
        else:
            slopes_array_s_jra55_fixed_poleward[j,k]=np.nan
            sigs_array_s_jra55_fixed_poleward[j,k]=np.nan

In [ ]:
dk=xr.Dataset()
dk['slopes_array_ivt_era5_variable']=(('lat','lon'),slopes_array_ivt_era5_variable.values)
dk['slopes_array_ivt_era5_fixed']=(('lat','lon'),slopes_array_ivt_era5_fixed.values)
dk['slopes_array_ivt_era5_fixed_poleward']=(('lat','lon'),slopes_array_ivt_era5_fixed_poleward.values)
dk['slopes_array_ivt_merra2_variable']=(('lat','lon'),slopes_array_ivt_merra2_variable.values)
dk['slopes_array_ivt_merra2_fixed']=(('lat','lon'),slopes_array_ivt_merra2_fixed.values)
dk['slopes_array_ivt_merra2_fixed_poleward']=(('lat','lon'),slopes_array_ivt_merra2_fixed_poleward.values)
dk['slopes_array_ivt_jra55_variable']=(('lat','lon'),slopes_array_ivt_jra55_variable.values)
dk['slopes_array_ivt_jra55_fixed']=(('lat','lon'),slopes_array_ivt_jra55_fixed.values)
dk['slopes_array_ivt_jra55_fixed_poleward']=(('lat','lon'),slopes_array_ivt_jra55_fixed_poleward.values)

dk['sigs_array_ivt_era5_variable']=(('lat','lon'),sigs_array_ivt_era5_variable.values)
dk['sigs_array_ivt_era5_fixed']=(('lat','lon'),sigs_array_ivt_era5_fixed.values)
dk['sigs_array_ivt_era5_fixed_poleward']=(('lat','lon'),sigs_array_ivt_era5_fixed_poleward.values)
dk['sigs_array_ivt_merra2_variable']=(('lat','lon'),sigs_array_ivt_merra2_variable.values)
dk['sigs_array_ivt_merra2_fixed']=(('lat','lon'),sigs_array_ivt_merra2_fixed.values)
dk['sigs_array_ivt_merra2_fixed_poleward']=(('lat','lon'),sigs_array_ivt_merra2_fixed_poleward.values)
dk['sigs_array_ivt_jra55_variable']=(('lat','lon'),sigs_array_ivt_jra55_variable.values)
dk['sigs_array_ivt_jra55_fixed']=(('lat','lon'),sigs_array_ivt_jra55_fixed.values)
dk['sigs_array_ivt_jra55_fixed_poleward']=(('lat','lon'),sigs_array_ivt_jra55_fixed_poleward.values)



dk['slopes_array_iwv_era5_variable']=(('lat','lon'),slopes_array_iwv_era5_variable.values)
dk['slopes_array_iwv_era5_fixed']=(('lat','lon'),slopes_array_iwv_era5_fixed.values)
dk['slopes_array_iwv_era5_fixed_poleward']=(('lat','lon'),slopes_array_iwv_era5_fixed_poleward.values)
dk['slopes_array_iwv_merra2_variable']=(('lat','lon'),slopes_array_iwv_merra2_variable.values)
dk['slopes_array_iwv_merra2_fixed']=(('lat','lon'),slopes_array_iwv_merra2_fixed.values)
dk['slopes_array_iwv_merra2_fixed_poleward']=(('lat','lon'),slopes_array_iwv_merra2_fixed_poleward.values)
dk['slopes_array_iwv_jra55_variable']=(('lat','lon'),slopes_array_iwv_jra55_variable.values)
dk['slopes_array_iwv_jra55_fixed']=(('lat','lon'),slopes_array_iwv_jra55_fixed.values)
dk['slopes_array_iwv_jra55_fixed_poleward']=(('lat','lon'),slopes_array_iwv_jra55_fixed_poleward.values)

dk['sigs_array_iwv_era5_variable']=(('lat','lon'),sigs_array_iwv_era5_variable.values)
dk['sigs_array_iwv_era5_fixed']=(('lat','lon'),sigs_array_iwv_era5_fixed.values)
dk['sigs_array_iwv_era5_fixed_poleward']=(('lat','lon'),sigs_array_iwv_era5_fixed_poleward.values)
dk['sigs_array_iwv_merra2_variable']=(('lat','lon'),sigs_array_iwv_merra2_variable.values)
dk['sigs_array_iwv_merra2_fixed']=(('lat','lon'),sigs_array_iwv_merra2_fixed.values)
dk['sigs_array_iwv_merra2_fixed_poleward']=(('lat','lon'),sigs_array_iwv_merra2_fixed_poleward.values)
dk['sigs_array_iwv_jra55_variable']=(('lat','lon'),sigs_array_iwv_jra55_variable.values)
dk['sigs_array_iwv_jra55_fixed']=(('lat','lon'),sigs_array_iwv_jra55_fixed.values)
dk['sigs_array_iwv_jra55_fixed_poleward']=(('lat','lon'),sigs_array_iwv_jra55_fixed_poleward.values)



dk['slopes_array_s_era5_variable']=(('lat','lon'),slopes_array_s_era5_variable.values)
dk['slopes_array_s_era5_fixed']=(('lat','lon'),slopes_array_s_era5_fixed.values)
dk['slopes_array_s_era5_fixed_poleward']=(('lat','lon'),slopes_array_s_era5_fixed_poleward.values)
dk['slopes_array_s_merra2_variable']=(('lat','lon'),slopes_array_s_merra2_variable.values)
dk['slopes_array_s_merra2_fixed']=(('lat','lon'),slopes_array_s_merra2_fixed.values)
dk['slopes_array_s_merra2_fixed_poleward']=(('lat','lon'),slopes_array_s_merra2_fixed_poleward.values)
dk['slopes_array_s_jra55_variable']=(('lat','lon'),slopes_array_s_jra55_variable.values)
dk['slopes_array_s_jra55_fixed']=(('lat','lon'),slopes_array_s_jra55_fixed.values)
dk['slopes_array_s_jra55_fixed_poleward']=(('lat','lon'),slopes_array_s_jra55_fixed_poleward.values)

dk['sigs_array_s_era5_variable']=(('lat','lon'),sigs_array_s_era5_variable.values)
dk['sigs_array_s_era5_fixed']=(('lat','lon'),sigs_array_s_era5_fixed.values)
dk['sigs_array_s_era5_fixed_poleward']=(('lat','lon'),sigs_array_s_era5_fixed_poleward.values)
dk['sigs_array_s_merra2_variable']=(('lat','lon'),sigs_array_s_merra2_variable.values)
dk['sigs_array_s_merra2_fixed']=(('lat','lon'),sigs_array_s_merra2_fixed.values)
dk['sigs_array_s_merra2_fixed_poleward']=(('lat','lon'),sigs_array_s_merra2_fixed_poleward.values)
dk['sigs_array_s_jra55_variable']=(('lat','lon'),sigs_array_s_jra55_variable.values)
dk['sigs_array_s_jra55_fixed']=(('lat','lon'),sigs_array_s_jra55_fixed.values)
dk['sigs_array_s_jra55_fixed_poleward']=(('lat','lon'),sigs_array_s_jra55_fixed_poleward.values)



dk['slopes_array_mfc_era5_variable']=(('lat','lon'),slopes_array_mfc_era5_variable.values)
dk['slopes_array_mfc_era5_fixed']=(('lat','lon'),slopes_array_mfc_era5_fixed.values)
dk['slopes_array_mfc_era5_fixed_poleward']=(('lat','lon'),slopes_array_mfc_era5_fixed_poleward.values)
dk['slopes_array_mfc_merra2_variable']=(('lat','lon'),slopes_array_mfc_merra2_variable.values)
dk['slopes_array_mfc_merra2_fixed']=(('lat','lon'),slopes_array_mfc_merra2_fixed.values)
dk['slopes_array_mfc_merra2_fixed_poleward']=(('lat','lon'),slopes_array_mfc_merra2_fixed_poleward.values)

dk['sigs_array_mfc_era5_variable']=(('lat','lon'),sigs_array_mfc_era5_variable.values)
dk['sigs_array_mfc_era5_fixed']=(('lat','lon'),sigs_array_mfc_era5_fixed.values)
dk['sigs_array_mfc_era5_fixed_poleward']=(('lat','lon'),sigs_array_mfc_era5_fixed_poleward.values)
dk['sigs_array_mfc_merra2_variable']=(('lat','lon'),sigs_array_mfc_merra2_variable.values)
dk['sigs_array_mfc_merra2_fixed']=(('lat','lon'),sigs_array_mfc_merra2_fixed.values)
dk['sigs_array_mfc_merra2_fixed_poleward']=(('lat','lon'),sigs_array_mfc_merra2_fixed_poleward.values)




dk.coords['lat']=slopes_array_ivt_era5_variable.lat.values
dk.coords['lon']=slopes_array_ivt_era5_variable.lon.values

dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'

try:
    os.remove(dir_data+'synoptic_geotrends_arrays_variants.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'synoptic_geotrends_arrays_variants.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'synoptic_geotrends_arrays_variants.nc')
slopes_array_ivt_era5_variable=ds['slopes_array_ivt_era5_variable']
slopes_array_ivt_era5_fixed=ds['slopes_array_ivt_era5_fixed']
slopes_array_ivt_era5_fixed_poleward=ds['slopes_array_ivt_era5_fixed_poleward']
slopes_array_ivt_merra2_variable=ds['slopes_array_ivt_merra2_variable']
slopes_array_ivt_merra2_fixed=ds['slopes_array_ivt_merra2_fixed']
slopes_array_ivt_merra2_fixed_poleward=ds['slopes_array_ivt_merra2_fixed_poleward']
slopes_array_ivt_jra55_variable=ds['slopes_array_ivt_jra55_variable']
slopes_array_ivt_jra55_fixed=ds['slopes_array_ivt_jra55_fixed']
slopes_array_ivt_jra55_fixed_poleward=ds['slopes_array_ivt_jra55_fixed_poleward']

sigs_array_ivt_era5_variable=ds['sigs_array_ivt_era5_variable']
sigs_array_ivt_era5_fixed=ds['sigs_array_ivt_era5_fixed']
sigs_array_ivt_era5_fixed_poleward=ds['sigs_array_ivt_era5_fixed_poleward']
sigs_array_ivt_merra2_variable=ds['sigs_array_ivt_merra2_variable']
sigs_array_ivt_merra2_fixed=ds['sigs_array_ivt_merra2_fixed']
sigs_array_ivt_merra2_fixed_poleward=ds['sigs_array_ivt_merra2_fixed_poleward']
sigs_array_ivt_jra55_variable=ds['sigs_array_ivt_jra55_variable']
sigs_array_ivt_jra55_fixed=ds['sigs_array_ivt_jra55_fixed']
sigs_array_ivt_jra55_fixed_poleward=ds['sigs_array_ivt_jra55_fixed_poleward']


slopes_array_iwv_era5_variable=ds['slopes_array_iwv_era5_variable']
slopes_array_iwv_era5_fixed=ds['slopes_array_iwv_era5_fixed']
slopes_array_iwv_era5_fixed_poleward=ds['slopes_array_iwv_era5_fixed_poleward']
slopes_array_iwv_merra2_variable=ds['slopes_array_iwv_merra2_variable']
slopes_array_iwv_merra2_fixed=ds['slopes_array_iwv_merra2_fixed']
slopes_array_iwv_merra2_fixed_poleward=ds['slopes_array_iwv_merra2_fixed_poleward']
slopes_array_iwv_jra55_variable=ds['slopes_array_iwv_jra55_variable']
slopes_array_iwv_jra55_fixed=ds['slopes_array_iwv_jra55_fixed']
slopes_array_iwv_jra55_fixed_poleward=ds['slopes_array_iwv_jra55_fixed_poleward']

sigs_array_iwv_era5_variable=ds['sigs_array_iwv_era5_variable']
sigs_array_iwv_era5_fixed=ds['sigs_array_iwv_era5_fixed']
sigs_array_iwv_era5_fixed_poleward=ds['sigs_array_iwv_era5_fixed_poleward']
sigs_array_iwv_merra2_variable=ds['sigs_array_iwv_merra2_variable']
sigs_array_iwv_merra2_fixed=ds['sigs_array_iwv_merra2_fixed']
sigs_array_iwv_merra2_fixed_poleward=ds['sigs_array_iwv_merra2_fixed_poleward']
sigs_array_iwv_jra55_variable=ds['sigs_array_iwv_jra55_variable']
sigs_array_iwv_jra55_fixed=ds['sigs_array_iwv_jra55_fixed']
sigs_array_iwv_jra55_fixed_poleward=ds['sigs_array_iwv_jra55_fixed_poleward']


slopes_array_s_era5_variable=ds['slopes_array_s_era5_variable']
slopes_array_s_era5_fixed=ds['slopes_array_s_era5_fixed']
slopes_array_s_era5_fixed_poleward=ds['slopes_array_s_era5_fixed_poleward']
slopes_array_s_merra2_variable=ds['slopes_array_s_merra2_variable']
slopes_array_s_merra2_fixed=ds['slopes_array_s_merra2_fixed']
slopes_array_s_merra2_fixed_poleward=ds['slopes_array_s_merra2_fixed_poleward']
slopes_array_s_jra55_variable=ds['slopes_array_s_jra55_variable']
slopes_array_s_jra55_fixed=ds['slopes_array_s_jra55_fixed']
slopes_array_s_jra55_fixed_poleward=ds['slopes_array_s_jra55_fixed_poleward']

sigs_array_s_era5_variable=ds['sigs_array_s_era5_variable']
sigs_array_s_era5_fixed=ds['sigs_array_s_era5_fixed']
sigs_array_s_era5_fixed_poleward=ds['sigs_array_s_era5_fixed_poleward']
sigs_array_s_merra2_variable=ds['sigs_array_s_merra2_variable']
sigs_array_s_merra2_fixed=ds['sigs_array_s_merra2_fixed']
sigs_array_s_merra2_fixed_poleward=ds['sigs_array_s_merra2_fixed_poleward']
sigs_array_s_jra55_variable=ds['sigs_array_s_jra55_variable']
sigs_array_s_jra55_fixed=ds['sigs_array_s_jra55_fixed']
sigs_array_s_jra55_fixed_poleward=ds['sigs_array_s_jra55_fixed_poleward']


slopes_array_mfc_era5_variable=ds['slopes_array_mfc_era5_variable']
slopes_array_mfc_era5_fixed=ds['slopes_array_mfc_era5_fixed']
slopes_array_mfc_era5_fixed_poleward=ds['slopes_array_mfc_era5_fixed_poleward']
slopes_array_mfc_merra2_variable=ds['slopes_array_mfc_merra2_variable']
slopes_array_mfc_merra2_fixed=ds['slopes_array_mfc_merra2_fixed']
slopes_array_mfc_merra2_fixed_poleward=ds['slopes_array_mfc_merra2_fixed_poleward']

sigs_array_mfc_era5_variable=ds['sigs_array_mfc_era5_variable']
sigs_array_mfc_era5_fixed=ds['sigs_array_mfc_era5_fixed']
sigs_array_mfc_era5_fixed_poleward=ds['sigs_array_mfc_era5_fixed_poleward']
sigs_array_mfc_merra2_variable=ds['sigs_array_mfc_merra2_variable']
sigs_array_mfc_merra2_fixed=ds['sigs_array_mfc_merra2_fixed']
sigs_array_mfc_merra2_fixed_poleward=ds['sigs_array_mfc_merra2_fixed_poleward']

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_ivt_merra2_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
print(N)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_ivt_merra2_variable=sigs_array_ivt_merra2_variable.where(sigs_array_ivt_merra2_variable<=p_final)

p_values=sigs_array_ivt_merra2_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
print(N)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_ivt_merra2_fixed=sigs_array_ivt_merra2_fixed.where(sigs_array_ivt_merra2_fixed<=p_final)

p_values=sigs_array_ivt_merra2_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_ivt_merra2_fixed_poleward=sigs_array_ivt_merra2_fixed_poleward.where(sigs_array_ivt_merra2_fixed_poleward<=p_final)

########

p_values=sigs_array_ivt_era5_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_ivt_era5_variable=sigs_array_ivt_era5_variable.where(sigs_array_ivt_era5_variable<=p_final)


p_values=sigs_array_ivt_era5_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_ivt_era5_fixed=sigs_array_ivt_era5_fixed.where(sigs_array_ivt_era5_fixed<=p_final)


p_values=sigs_array_ivt_era5_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_ivt_era5_fixed_poleward=sigs_array_ivt_era5_fixed_poleward.where(sigs_array_ivt_era5_fixed_poleward<=p_final)

############

p_values=sigs_array_ivt_jra55_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_ivt_jra55_variable=sigs_array_ivt_jra55_variable.where(sigs_array_ivt_jra55_variable<=p_final)


p_values=sigs_array_ivt_jra55_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_ivt_jra55_fixed=sigs_array_ivt_jra55_fixed.where(sigs_array_ivt_jra55_fixed<=p_final)


p_values=sigs_array_ivt_jra55_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_ivt_jra55_fixed_poleward=sigs_array_ivt_jra55_fixed_poleward.where(sigs_array_ivt_jra55_fixed_poleward<=p_final)

In [ ]:
num_sets1=xr.ones_like(slopes_array_ivt_merra2_variable).where(slopes_array_ivt_merra2_variable>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_ivt_merra2_fixed).where(slopes_array_ivt_merra2_fixed>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_ivt_merra2_fixed_poleward).where(slopes_array_ivt_merra2_fixed_poleward>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_ivt_era5_variable).where(slopes_array_ivt_era5_variable>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_ivt_era5_fixed).where(slopes_array_ivt_era5_fixed>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_ivt_era5_fixed_poleward).where(slopes_array_ivt_era5_fixed_poleward>-1000)
num_sets6=num_sets6.fillna(0)

num_sets7=xr.ones_like(slopes_array_ivt_jra55_variable).where(slopes_array_ivt_jra55_variable>-1000)
num_sets7=num_sets7.fillna(0)
num_sets8=xr.ones_like(slopes_array_ivt_jra55_fixed).where(slopes_array_ivt_jra55_fixed>-1000)
num_sets8=num_sets8.fillna(0)
num_sets9=xr.ones_like(slopes_array_ivt_jra55_fixed_poleward).where(slopes_array_ivt_jra55_fixed_poleward>-1000)
num_sets9=num_sets9.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6+num_sets7+num_sets8+num_sets9
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,10,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
test1=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_ivt_merra2_variable<=0.05).where(slopes_array_ivt_merra2_variable>0).fillna(0)
test2=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_ivt_merra2_fixed<=0.05).where(slopes_array_ivt_merra2_fixed>0).fillna(0)
test3=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_ivt_merra2_fixed_poleward<=0.05).where(slopes_array_ivt_merra2_fixed_poleward>0).fillna(0)
test4=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_ivt_era5_variable<=0.05).where(slopes_array_ivt_era5_variable>0).fillna(0)
test5=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_ivt_era5_fixed<=0.05).where(slopes_array_ivt_era5_fixed>0).fillna(0)
test6=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_ivt_era5_fixed_poleward<=0.05).where(slopes_array_ivt_era5_fixed_poleward>0).fillna(0)
test7=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_ivt_jra55_variable<=0.05).where(slopes_array_ivt_jra55_variable>0).fillna(0)
test8=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_ivt_jra55_fixed<=0.05).where(slopes_array_ivt_jra55_fixed>0).fillna(0)
test9=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_ivt_jra55_fixed_poleward<=0.05).where(slopes_array_ivt_jra55_fixed_poleward>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7+test8+test9)/num_sets*100.
field=test_all

test1a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_ivt_merra2_variable<=0.05).where(slopes_array_ivt_merra2_variable<0).fillna(0)
test2a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_ivt_merra2_fixed<=0.05).where(slopes_array_ivt_merra2_fixed<0).fillna(0)
test3a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_ivt_merra2_fixed_poleward<=0.05).where(slopes_array_ivt_merra2_fixed_poleward<0).fillna(0)
test4a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_ivt_era5_variable<=0.05).where(slopes_array_ivt_era5_variable<0).fillna(0)
test5a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_ivt_era5_fixed<=0.05).where(slopes_array_ivt_era5_fixed<0).fillna(0)
test6a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_ivt_era5_fixed_poleward<=0.05).where(slopes_array_ivt_era5_fixed_poleward<0).fillna(0)
test7a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_ivt_jra55_variable<=0.05).where(slopes_array_ivt_jra55_variable<0).fillna(0)
test8a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_ivt_jra55_fixed<=0.05).where(slopes_array_ivt_jra55_fixed<0).fillna(0)
test9a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_ivt_jra55_fixed_poleward<=0.05).where(slopes_array_ivt_jra55_fixed_poleward<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a+test8a+test9a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

############

test1b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_variable<=0.05).where(slopes_array_ivt_merra2_variable>0).fillna(0)
test2b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_fixed<=0.05).where(slopes_array_ivt_merra2_fixed>0).fillna(0)
test3b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_fixed_poleward<=0.05).where(slopes_array_ivt_merra2_fixed_poleward>0).fillna(0)
test4b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_variable<=0.05).where(slopes_array_ivt_era5_variable>0).fillna(0)
test5b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_fixed<=0.05).where(slopes_array_ivt_era5_fixed>0).fillna(0)
test6b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_fixed_poleward<=0.05).where(slopes_array_ivt_era5_fixed_poleward>0).fillna(0)
test7b=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_variable<=0.05).where(slopes_array_ivt_jra55_variable>0).fillna(0)
test8b=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_fixed<=0.05).where(slopes_array_ivt_jra55_fixed>0).fillna(0)
test9b=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_fixed_poleward<=0.05).where(slopes_array_ivt_jra55_fixed_poleward>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b+test8b+test9b)/num_sets*100.

test1c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_variable<=0.05).where(slopes_array_ivt_merra2_variable<0).fillna(0)
test2c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_fixed<=0.05).where(slopes_array_ivt_merra2_fixed<0).fillna(0)
test3c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_fixed_poleward<=0.05).where(slopes_array_ivt_merra2_fixed_poleward<0).fillna(0)
test4c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_variable<=0.05).where(slopes_array_ivt_era5_variable<0).fillna(0)
test5c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_fixed<=0.05).where(slopes_array_ivt_era5_fixed<0).fillna(0)
test6c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_fixed_poleward<=0.05).where(slopes_array_ivt_era5_fixed_poleward<0).fillna(0)
test7c=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_variable<=0.05).where(slopes_array_ivt_jra55_variable<0).fillna(0)
test8c=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_fixed<=0.05).where(slopes_array_ivt_jra55_fixed<0).fillna(0)
test9c=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_fixed_poleward<=0.05).where(slopes_array_ivt_jra55_fixed_poleward<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c+test8c+test9c)/num_sets*100.

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])
cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1,levels=np.arange(-100,120,20),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{trends:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{IVT}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_variants_1.png')

In [ ]:
test1=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_variable<=0.05).where(slopes_array_ivt_merra2_variable>0).fillna(0)
test2=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_fixed<=0.05).where(slopes_array_ivt_merra2_fixed>0).fillna(0)
test3=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_fixed_poleward<=0.05).where(slopes_array_ivt_merra2_fixed_poleward>0).fillna(0)
test4=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_variable<=0.05).where(slopes_array_ivt_era5_variable>0).fillna(0)
test5=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_fixed<=0.05).where(slopes_array_ivt_era5_fixed>0).fillna(0)
test6=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_fixed_poleward<=0.05).where(slopes_array_ivt_era5_fixed_poleward>0).fillna(0)
test7=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_variable<=0.05).where(slopes_array_ivt_jra55_variable>0).fillna(0)
test8=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_fixed<=0.05).where(slopes_array_ivt_jra55_fixed>0).fillna(0)
test9=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_fixed_poleward<=0.05).where(slopes_array_ivt_jra55_fixed_poleward>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7+test8+test9)/num_sets*100.
field=test_all

test1a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_variable<=0.05).where(slopes_array_ivt_merra2_variable<0).fillna(0)
test2a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_fixed<=0.05).where(slopes_array_ivt_merra2_fixed<0).fillna(0)
test3a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_ivt_merra2_fixed_poleward<=0.05).where(slopes_array_ivt_merra2_fixed_poleward<0).fillna(0)
test4a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_variable<=0.05).where(slopes_array_ivt_era5_variable<0).fillna(0)
test5a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_fixed<=0.05).where(slopes_array_ivt_era5_fixed<0).fillna(0)
test6a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_ivt_era5_fixed_poleward<=0.05).where(slopes_array_ivt_era5_fixed_poleward<0).fillna(0)
test7a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_variable<=0.05).where(slopes_array_ivt_jra55_variable<0).fillna(0)
test8a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_fixed<=0.05).where(slopes_array_ivt_jra55_fixed<0).fillna(0)
test9a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_ivt_jra55_fixed_poleward<=0.05).where(slopes_array_ivt_jra55_fixed_poleward<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a+test8a+test9a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])
cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1,levels=np.arange(-100,120,20),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{trends:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{IVT}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_variants_1a.png')

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_iwv_merra2_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_iwv_merra2_variable=sigs_array_iwv_merra2_variable.where(sigs_array_iwv_merra2_variable<=p_final)

p_values=sigs_array_iwv_merra2_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_iwv_merra2_fixed=sigs_array_iwv_merra2_fixed.where(sigs_array_iwv_merra2_fixed<=p_final)

p_values=sigs_array_iwv_merra2_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_iwv_merra2_fixed_poleward=sigs_array_iwv_merra2_fixed_poleward.where(sigs_array_iwv_merra2_fixed_poleward<=p_final)

########

p_values=sigs_array_iwv_era5_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_iwv_era5_variable=sigs_array_iwv_era5_variable.where(sigs_array_iwv_era5_variable<=p_final)


p_values=sigs_array_iwv_era5_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_iwv_era5_fixed=sigs_array_iwv_era5_fixed.where(sigs_array_iwv_era5_fixed<=p_final)


p_values=sigs_array_iwv_era5_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_iwv_era5_fixed_poleward=sigs_array_iwv_era5_fixed_poleward.where(sigs_array_iwv_era5_fixed_poleward<=p_final)

############

p_values=sigs_array_iwv_jra55_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_iwv_jra55_variable=sigs_array_iwv_jra55_variable.where(sigs_array_iwv_jra55_variable<=p_final)


p_values=sigs_array_iwv_jra55_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_iwv_jra55_fixed=sigs_array_iwv_jra55_fixed.where(sigs_array_iwv_jra55_fixed<=p_final)


p_values=sigs_array_iwv_jra55_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_iwv_jra55_fixed_poleward=sigs_array_iwv_jra55_fixed_poleward.where(sigs_array_iwv_jra55_fixed_poleward<=p_final)

In [ ]:
test1=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_iwv_merra2_variable<=0.05).where(slopes_array_iwv_merra2_variable>0).fillna(0)
test2=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_iwv_merra2_fixed<=0.05).where(slopes_array_iwv_merra2_fixed>0).fillna(0)
test3=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_iwv_merra2_fixed_poleward<=0.05).where(slopes_array_iwv_merra2_fixed_poleward>0).fillna(0)
test4=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_iwv_era5_variable<=0.05).where(slopes_array_iwv_era5_variable>0).fillna(0)
test5=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_iwv_era5_fixed<=0.05).where(slopes_array_iwv_era5_fixed>0).fillna(0)
test6=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_iwv_era5_fixed_poleward<=0.05).where(slopes_array_iwv_era5_fixed_poleward>0).fillna(0)
test7=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_iwv_jra55_variable<=0.05).where(slopes_array_iwv_jra55_variable>0).fillna(0)
test8=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_iwv_jra55_fixed<=0.05).where(slopes_array_iwv_jra55_fixed>0).fillna(0)
test9=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_iwv_jra55_fixed_poleward<=0.05).where(slopes_array_iwv_jra55_fixed_poleward>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7+test8+test9)/num_sets*100.
field=test_all

test1a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_iwv_merra2_variable<=0.05).where(slopes_array_iwv_merra2_variable<0).fillna(0)
test2a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_iwv_merra2_fixed<=0.05).where(slopes_array_iwv_merra2_fixed<0).fillna(0)
test3a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_iwv_merra2_fixed_poleward<=0.05).where(slopes_array_iwv_merra2_fixed_poleward<0).fillna(0)
test4a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_iwv_era5_variable<=0.05).where(slopes_array_iwv_era5_variable<0).fillna(0)
test5a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_iwv_era5_fixed<=0.05).where(slopes_array_iwv_era5_fixed<0).fillna(0)
test6a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_iwv_era5_fixed_poleward<=0.05).where(slopes_array_iwv_era5_fixed_poleward<0).fillna(0)
test7a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_iwv_jra55_variable<=0.05).where(slopes_array_iwv_jra55_variable<0).fillna(0)
test8a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_iwv_jra55_fixed<=0.05).where(slopes_array_iwv_jra55_fixed<0).fillna(0)
test9a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_iwv_jra55_fixed_poleward<=0.05).where(slopes_array_iwv_jra55_fixed_poleward<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a+test8a+test9a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

############

test1b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_variable<=0.05).where(slopes_array_iwv_merra2_variable>0).fillna(0)
test2b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_fixed<=0.05).where(slopes_array_iwv_merra2_fixed>0).fillna(0)
test3b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_fixed_poleward<=0.05).where(slopes_array_iwv_merra2_fixed_poleward>0).fillna(0)
test4b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_variable<=0.05).where(slopes_array_iwv_era5_variable>0).fillna(0)
test5b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_fixed<=0.05).where(slopes_array_iwv_era5_fixed>0).fillna(0)
test6b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_fixed_poleward<=0.05).where(slopes_array_iwv_era5_fixed_poleward>0).fillna(0)
test7b=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_variable<=0.05).where(slopes_array_iwv_jra55_variable>0).fillna(0)
test8b=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_fixed<=0.05).where(slopes_array_iwv_jra55_fixed>0).fillna(0)
test9b=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_fixed_poleward<=0.05).where(slopes_array_iwv_jra55_fixed_poleward>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b+test8b+test9b)/num_sets*100.

test1c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_variable<=0.05).where(slopes_array_iwv_merra2_variable<0).fillna(0)
test2c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_fixed<=0.05).where(slopes_array_iwv_merra2_fixed<0).fillna(0)
test3c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_fixed_poleward<=0.05).where(slopes_array_iwv_merra2_fixed_poleward<0).fillna(0)
test4c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_variable<=0.05).where(slopes_array_iwv_era5_variable<0).fillna(0)
test5c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_fixed<=0.05).where(slopes_array_iwv_era5_fixed<0).fillna(0)
test6c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_fixed_poleward<=0.05).where(slopes_array_iwv_era5_fixed_poleward<0).fillna(0)
test7c=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_variable<=0.05).where(slopes_array_iwv_jra55_variable<0).fillna(0)
test8c=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_fixed<=0.05).where(slopes_array_iwv_jra55_fixed<0).fillna(0)
test9c=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_fixed_poleward<=0.05).where(slopes_array_iwv_jra55_fixed_poleward<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c+test8c+test9c)/num_sets*100.

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])
cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1,levels=np.arange(-100,120,20),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{trends:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{IWV}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_variants_2.png')

In [ ]:
test1=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_variable<=0.05).where(slopes_array_iwv_merra2_variable>0).fillna(0)
test2=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_fixed<=0.05).where(slopes_array_iwv_merra2_fixed>0).fillna(0)
test3=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_fixed_poleward<=0.05).where(slopes_array_iwv_merra2_fixed_poleward>0).fillna(0)
test4=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_variable<=0.05).where(slopes_array_iwv_era5_variable>0).fillna(0)
test5=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_fixed<=0.05).where(slopes_array_iwv_era5_fixed>0).fillna(0)
test6=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_fixed_poleward<=0.05).where(slopes_array_iwv_era5_fixed_poleward>0).fillna(0)
test7=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_variable<=0.05).where(slopes_array_iwv_jra55_variable>0).fillna(0)
test8=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_fixed<=0.05).where(slopes_array_iwv_jra55_fixed>0).fillna(0)
test9=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_fixed_poleward<=0.05).where(slopes_array_iwv_jra55_fixed_poleward>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7+test8+test9)/num_sets*100.
field=test_all

test1a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_variable<=0.05).where(slopes_array_iwv_merra2_variable<0).fillna(0)
test2a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_fixed<=0.05).where(slopes_array_iwv_merra2_fixed<0).fillna(0)
test3a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_iwv_merra2_fixed_poleward<=0.05).where(slopes_array_iwv_merra2_fixed_poleward<0).fillna(0)
test4a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_variable<=0.05).where(slopes_array_iwv_era5_variable<0).fillna(0)
test5a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_fixed<=0.05).where(slopes_array_iwv_era5_fixed<0).fillna(0)
test6a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_iwv_era5_fixed_poleward<=0.05).where(slopes_array_iwv_era5_fixed_poleward<0).fillna(0)
test7a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_variable<=0.05).where(slopes_array_iwv_jra55_variable<0).fillna(0)
test8a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_fixed<=0.05).where(slopes_array_iwv_jra55_fixed<0).fillna(0)
test9a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_iwv_jra55_fixed_poleward<=0.05).where(slopes_array_iwv_jra55_fixed_poleward<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a+test8a+test9a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])
cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1,levels=np.arange(-100,120,20),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{trends:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{IWV}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_variants_2a.png')

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_s_merra2_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_s_merra2_variable=sigs_array_s_merra2_variable.where(sigs_array_s_merra2_variable<=p_final)

p_values=sigs_array_s_merra2_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_s_merra2_fixed=sigs_array_s_merra2_fixed.where(sigs_array_s_merra2_fixed<=p_final)

p_values=sigs_array_s_merra2_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_s_merra2_fixed_poleward=sigs_array_s_merra2_fixed_poleward.where(sigs_array_s_merra2_fixed_poleward<=p_final)

########

p_values=sigs_array_s_era5_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_s_era5_variable=sigs_array_s_era5_variable.where(sigs_array_s_era5_variable<=p_final)


p_values=sigs_array_s_era5_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_s_era5_fixed=sigs_array_s_era5_fixed.where(sigs_array_s_era5_fixed<=p_final)


p_values=sigs_array_s_era5_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_s_era5_fixed_poleward=sigs_array_s_era5_fixed_poleward.where(sigs_array_s_era5_fixed_poleward<=p_final)

############

p_values=sigs_array_s_jra55_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_s_jra55_variable=sigs_array_s_jra55_variable.where(sigs_array_s_jra55_variable<=p_final)


p_values=sigs_array_s_jra55_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_s_jra55_fixed=sigs_array_s_jra55_fixed.where(sigs_array_s_jra55_fixed<=p_final)


p_values=sigs_array_s_jra55_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_s_jra55_fixed_poleward=sigs_array_s_jra55_fixed_poleward.where(sigs_array_s_jra55_fixed_poleward<=p_final)

In [ ]:
num_sets1=xr.ones_like(slopes_array_ivt_merra2_variable).where(slopes_array_s_merra2_variable>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_ivt_merra2_fixed).where(slopes_array_s_merra2_fixed>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_ivt_merra2_fixed_poleward).where(slopes_array_s_merra2_fixed_poleward>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_ivt_era5_variable).where(slopes_array_s_era5_variable>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_ivt_era5_fixed).where(slopes_array_s_era5_fixed>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_ivt_era5_fixed_poleward).where(slopes_array_s_era5_fixed_poleward>-1000)
num_sets6=num_sets6.fillna(0)

num_sets7=xr.ones_like(slopes_array_ivt_jra55_variable).where(slopes_array_s_jra55_variable>-1000)
num_sets7=num_sets7.fillna(0)
num_sets8=xr.ones_like(slopes_array_ivt_jra55_fixed).where(slopes_array_s_jra55_fixed>-1000)
num_sets8=num_sets8.fillna(0)
num_sets9=xr.ones_like(slopes_array_ivt_jra55_fixed_poleward).where(slopes_array_s_jra55_fixed_poleward>-1000)
num_sets9=num_sets9.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6+num_sets7+num_sets8+num_sets9
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,10,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
test1=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_s_merra2_variable<=0.05).where(slopes_array_s_merra2_variable>0).fillna(0)
test2=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_s_merra2_fixed<=0.05).where(slopes_array_s_merra2_fixed>0).fillna(0)
test3=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_s_merra2_fixed_poleward<=0.05).where(slopes_array_s_merra2_fixed_poleward>0).fillna(0)
test4=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_s_era5_variable<=0.05).where(slopes_array_s_era5_variable>0).fillna(0)
test5=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_s_era5_fixed<=0.05).where(slopes_array_s_era5_fixed>0).fillna(0)
test6=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_s_era5_fixed_poleward<=0.05).where(slopes_array_s_era5_fixed_poleward>0).fillna(0)
test7=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_s_jra55_variable<=0.05).where(slopes_array_s_jra55_variable>0).fillna(0)
test8=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_s_jra55_fixed<=0.05).where(slopes_array_s_jra55_fixed>0).fillna(0)
test9=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_s_jra55_fixed_poleward<=0.05).where(slopes_array_s_jra55_fixed_poleward>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7+test8+test9)/num_sets*100.
field=test_all

test1a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_s_merra2_variable<=0.05).where(slopes_array_s_merra2_variable<0).fillna(0)
test2a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_s_merra2_fixed<=0.05).where(slopes_array_s_merra2_fixed<0).fillna(0)
test3a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_s_merra2_fixed_poleward<=0.05).where(slopes_array_s_merra2_fixed_poleward<0).fillna(0)
test4a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_s_era5_variable<=0.05).where(slopes_array_s_era5_variable<0).fillna(0)
test5a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_s_era5_fixed<=0.05).where(slopes_array_s_era5_fixed<0).fillna(0)
test6a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_s_era5_fixed_poleward<=0.05).where(slopes_array_s_era5_fixed_poleward<0).fillna(0)
test7a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_s_jra55_variable<=0.05).where(slopes_array_s_jra55_variable<0).fillna(0)
test8a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_s_jra55_fixed<=0.05).where(slopes_array_s_jra55_fixed<0).fillna(0)
test9a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_final_s_jra55_fixed_poleward<=0.05).where(slopes_array_s_jra55_fixed_poleward<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a+test8a+test9a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

############

test1b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_variable<=0.05).where(slopes_array_s_merra2_variable>0).fillna(0)
test2b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_fixed<=0.05).where(slopes_array_s_merra2_fixed>0).fillna(0)
test3b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_fixed_poleward<=0.05).where(slopes_array_s_merra2_fixed_poleward>0).fillna(0)
test4b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_variable<=0.05).where(slopes_array_s_era5_variable>0).fillna(0)
test5b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_fixed<=0.05).where(slopes_array_s_era5_fixed>0).fillna(0)
test6b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_fixed_poleward<=0.05).where(slopes_array_s_era5_fixed_poleward>0).fillna(0)
test7b=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_variable<=0.05).where(slopes_array_s_jra55_variable>0).fillna(0)
test8b=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_fixed<=0.05).where(slopes_array_s_jra55_fixed>0).fillna(0)
test9b=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_fixed_poleward<=0.05).where(slopes_array_s_jra55_fixed_poleward>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b+test7b+test8b+test9b)/num_sets*100.

test1c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_variable<=0.05).where(slopes_array_s_merra2_variable<0).fillna(0)
test2c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_fixed<=0.05).where(slopes_array_s_merra2_fixed<0).fillna(0)
test3c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_fixed_poleward<=0.05).where(slopes_array_s_merra2_fixed_poleward<0).fillna(0)
test4c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_variable<=0.05).where(slopes_array_s_era5_variable<0).fillna(0)
test5c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_fixed<=0.05).where(slopes_array_s_era5_fixed<0).fillna(0)
test6c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_fixed_poleward<=0.05).where(slopes_array_s_era5_fixed_poleward<0).fillna(0)
test7c=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_variable<=0.05).where(slopes_array_s_jra55_variable<0).fillna(0)
test8c=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_fixed<=0.05).where(slopes_array_s_jra55_fixed<0).fillna(0)
test9c=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_fixed_poleward<=0.05).where(slopes_array_s_jra55_fixed_poleward<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c+test7c+test8c+test9c)/num_sets*100.

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])
cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1,levels=np.arange(-100,120,20),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{trends:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{S}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_variants_3.png')

In [ ]:
test1=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_variable<=0.05).where(slopes_array_s_merra2_variable>0).fillna(0)
test2=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_fixed<=0.05).where(slopes_array_s_merra2_fixed>0).fillna(0)
test3=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_fixed_poleward<=0.05).where(slopes_array_s_merra2_fixed_poleward>0).fillna(0)
test4=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_variable<=0.05).where(slopes_array_s_era5_variable>0).fillna(0)
test5=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_fixed<=0.05).where(slopes_array_s_era5_fixed>0).fillna(0)
test6=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_fixed_poleward<=0.05).where(slopes_array_s_era5_fixed_poleward>0).fillna(0)
test7=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_variable<=0.05).where(slopes_array_s_jra55_variable>0).fillna(0)
test8=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_fixed<=0.05).where(slopes_array_s_jra55_fixed>0).fillna(0)
test9=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_fixed_poleward<=0.05).where(slopes_array_s_jra55_fixed_poleward>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6+test7+test8+test9)/num_sets*100.
field=test_all

test1a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_variable<=0.05).where(slopes_array_s_merra2_variable<0).fillna(0)
test2a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_fixed<=0.05).where(slopes_array_s_merra2_fixed<0).fillna(0)
test3a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_s_merra2_fixed_poleward<=0.05).where(slopes_array_s_merra2_fixed_poleward<0).fillna(0)
test4a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_variable<=0.05).where(slopes_array_s_era5_variable<0).fillna(0)
test5a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_fixed<=0.05).where(slopes_array_s_era5_fixed<0).fillna(0)
test6a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_s_era5_fixed_poleward<=0.05).where(slopes_array_s_era5_fixed_poleward<0).fillna(0)
test7a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_variable<=0.05).where(slopes_array_s_jra55_variable<0).fillna(0)
test8a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_fixed<=0.05).where(slopes_array_s_jra55_fixed<0).fillna(0)
test9a=xr.ones_like(sigs_array_final_ivt_jra55_variable).where(sigs_array_s_jra55_fixed_poleward<=0.05).where(slopes_array_s_jra55_fixed_poleward<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a+test7a+test8a+test9a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])
cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1,levels=np.arange(-100,120,20),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{trends:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{S}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_variants_3a.png')

In [ ]:
num_sets1=xr.ones_like(slopes_array_mfc_merra2_variable).where(slopes_array_mfc_merra2_variable>-1000)
num_sets1=num_sets1.fillna(0)
num_sets2=xr.ones_like(slopes_array_mfc_merra2_fixed).where(slopes_array_mfc_merra2_fixed>-1000)
num_sets2=num_sets2.fillna(0)
num_sets3=xr.ones_like(slopes_array_mfc_merra2_fixed_poleward).where(slopes_array_mfc_merra2_fixed_poleward>-1000)
num_sets3=num_sets3.fillna(0)

num_sets4=xr.ones_like(slopes_array_mfc_era5_variable).where(slopes_array_mfc_era5_variable>-1000)
num_sets4=num_sets4.fillna(0)
num_sets5=xr.ones_like(slopes_array_mfc_era5_fixed).where(slopes_array_mfc_era5_fixed>-1000)
num_sets5=num_sets5.fillna(0)
num_sets6=xr.ones_like(slopes_array_mfc_era5_fixed_poleward).where(slopes_array_mfc_era5_fixed_poleward>-1000)
num_sets6=num_sets6.fillna(0)

num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6
field=num_sets


fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,10,1),cmap=plt.cm.Greens,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

#cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),cmap=plt.cm.seismic,extend='both',transform=ccrs.PlateCarree(),alpha=1.0)   


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('# datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False
#ax.set_ylim(-60,70)

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{\#}$'+' '+r'$\bf{datasets:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{AR}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)

plt.show()

In [ ]:
a=0.05#/sum_factor

p_values=sigs_array_mfc_merra2_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_mfc_merra2_variable=sigs_array_mfc_merra2_variable.where(sigs_array_mfc_merra2_variable<=p_final)

p_values=sigs_array_mfc_merra2_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_mfc_merra2_fixed=sigs_array_mfc_merra2_fixed.where(sigs_array_mfc_merra2_fixed<=p_final)

p_values=sigs_array_mfc_merra2_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final=max(pass_list)
print(p_final)
#p_final=0.05
sigs_array_final_mfc_merra2_fixed_poleward=sigs_array_mfc_merra2_fixed_poleward.where(sigs_array_mfc_merra2_fixed_poleward<=p_final)

########

p_values=sigs_array_mfc_era5_variable.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)

pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_mfc_era5_variable=sigs_array_mfc_era5_variable.where(sigs_array_mfc_era5_variable<=p_final)


p_values=sigs_array_mfc_era5_fixed.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_mfc_era5_fixed=sigs_array_mfc_era5_fixed.where(sigs_array_mfc_era5_fixed<=p_final)


p_values=sigs_array_mfc_era5_fixed_poleward.values.flatten().tolist()
p_values=[x for x in p_values if x>=0]
p_values=sorted(p_values)
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
try:
    p_final=max(pass_list)
except:
    p_final=0
print(p_final)
#p_final=0.05
sigs_array_final_mfc_era5_fixed_poleward=sigs_array_mfc_era5_fixed_poleward.where(sigs_array_mfc_era5_fixed_poleward<=p_final)

In [ ]:
test1=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_mfc_merra2_variable<=0.05).where(slopes_array_mfc_merra2_variable>0).fillna(0)
test2=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_mfc_merra2_fixed<=0.05).where(slopes_array_mfc_merra2_fixed>0).fillna(0)
test3=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_mfc_merra2_fixed_poleward<=0.05).where(slopes_array_mfc_merra2_fixed_poleward>0).fillna(0)
test4=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_mfc_era5_variable<=0.05).where(slopes_array_mfc_era5_variable>0).fillna(0)
test5=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_mfc_era5_fixed<=0.05).where(slopes_array_mfc_era5_fixed>0).fillna(0)
test6=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_mfc_era5_fixed_poleward<=0.05).where(slopes_array_mfc_era5_fixed_poleward>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6)/num_sets*100.
field=test_all

test1a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_mfc_merra2_variable<=0.05).where(slopes_array_mfc_merra2_variable<0).fillna(0)
test2a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_mfc_merra2_fixed<=0.05).where(slopes_array_mfc_merra2_fixed<0).fillna(0)
test3a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_final_mfc_merra2_fixed_poleward<=0.05).where(slopes_array_mfc_merra2_fixed_poleward<0).fillna(0)
test4a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_mfc_era5_variable<=0.05).where(slopes_array_mfc_era5_variable<0).fillna(0)
test5a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_mfc_era5_fixed<=0.05).where(slopes_array_mfc_era5_fixed<0).fillna(0)
test6a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_final_mfc_era5_fixed_poleward<=0.05).where(slopes_array_mfc_era5_fixed_poleward<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

############

test1b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_variable<=0.05).where(slopes_array_mfc_merra2_variable>0).fillna(0)
test2b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_fixed<=0.05).where(slopes_array_mfc_merra2_fixed>0).fillna(0)
test3b=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_fixed_poleward<=0.05).where(slopes_array_mfc_merra2_fixed_poleward>0).fillna(0)
test4b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_variable<=0.05).where(slopes_array_mfc_era5_variable>0).fillna(0)
test5b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_fixed<=0.05).where(slopes_array_mfc_era5_fixed>0).fillna(0)
test6b=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_fixed_poleward<=0.05).where(slopes_array_mfc_era5_fixed_poleward>0).fillna(0)

test_allb=(test1b+test2b+test3b+test4b+test5b+test6b)/num_sets*100.

test1c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_variable<=0.05).where(slopes_array_mfc_merra2_variable<0).fillna(0)
test2c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_fixed<=0.05).where(slopes_array_mfc_merra2_fixed<0).fillna(0)
test3c=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_fixed_poleward<=0.05).where(slopes_array_mfc_merra2_fixed_poleward<0).fillna(0)
test4c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_variable<=0.05).where(slopes_array_mfc_era5_variable<0).fillna(0)
test5c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_fixed<=0.05).where(slopes_array_mfc_era5_fixed<0).fillna(0)
test6c=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_fixed_poleward<=0.05).where(slopes_array_mfc_era5_fixed_poleward<0).fillna(0)

test_allc=(test1c+test2c+test3c+test4c+test5c+test6c)/num_sets*100.

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)
test_allb=test_allb.where(num_sets>=2)
test_allc=test_allc.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])
cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1,levels=np.arange(-100,120,20),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

X=4
for j in range(int(field.lat.size/X)):
    for k in range(int(field.lon.size/X)):
        if test_allb[X*j,X*k]>=66.66 or test_allc[X*j,X*k]>=66.66:
            ax.plot(field.lon[X*k],field.lat[X*j],marker='o',color='k',markersize=2,alpha=0.5)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{trends:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{MFC}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_variants_4.png')

In [ ]:
test1=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_variable<=0.05).where(slopes_array_mfc_merra2_variable>0).fillna(0)
test2=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_fixed<=0.05).where(slopes_array_mfc_merra2_fixed>0).fillna(0)
test3=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_fixed_poleward<=0.05).where(slopes_array_mfc_merra2_fixed_poleward>0).fillna(0)
test4=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_variable<=0.05).where(slopes_array_mfc_era5_variable>0).fillna(0)
test5=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_fixed<=0.05).where(slopes_array_mfc_era5_fixed>0).fillna(0)
test6=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_fixed_poleward<=0.05).where(slopes_array_mfc_era5_fixed_poleward>0).fillna(0)

test_all=(test1+test2+test3+test4+test5+test6)/num_sets*100.
field=test_all

test1a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_variable<=0.05).where(slopes_array_mfc_merra2_variable<0).fillna(0)
test2a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_fixed<=0.05).where(slopes_array_mfc_merra2_fixed<0).fillna(0)
test3a=xr.ones_like(sigs_array_final_ivt_merra2_variable).where(sigs_array_mfc_merra2_fixed_poleward<=0.05).where(slopes_array_mfc_merra2_fixed_poleward<0).fillna(0)
test4a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_variable<=0.05).where(slopes_array_mfc_era5_variable<0).fillna(0)
test5a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_fixed<=0.05).where(slopes_array_mfc_era5_fixed<0).fillna(0)
test6a=xr.ones_like(sigs_array_final_ivt_era5_variable).where(sigs_array_mfc_era5_fixed_poleward<=0.05).where(slopes_array_mfc_era5_fixed_poleward<0).fillna(0)

test_alla=(test1a+test2a+test3a+test4a+test5a+test6a)/num_sets*100.
fielda=test_alla

fieldb=field.where(field==fielda)
fieldb=fieldb/fieldb
field=field.where(field>fielda).fillna(0)
fielda=fielda.where(fielda>field)

field=field.where(num_sets>=2)
fielda=fielda.where(num_sets>=2)

from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(-100,110,10),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(-100,120,20))
cbar.ax.set_yticklabels([' ','80','60','40','20','0','20','40','60','80',' '])
cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1,levels=np.arange(-100,120,20),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',['w','magenta'])
cax=ax.pcolormesh(fieldb.lon,fieldb.lat,fieldb,vmin=0,vmax=1.0,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=32) 
cbar.set_label('% datasets',fontsize=36,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':28,'color':'k'}
g1.ylabel_style={'size':28,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
#ax.set_title(r'$\bf{IWV}$'+' '+r'$\bf{trends:}$'+' '+r'$\bf{3}$'+' '+r'$\bf{variants}$',fontsize=40,pad=5)
ax.set_title(r'$\bf{MFC}$',fontsize=40,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'artmip_geotrends_redo_variants_4a.png')